### Start and Run All

In [2]:
import os
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
# from datetime import date, timedelta
from datetime import datetime
from time import sleep, strftime

# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2026-01-05 21:11:19


In [3]:
url = "https://www.set.or.th/en/market/product/stock/quote/"
percent_span = '' 

In [4]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [5]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"Obsidian path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A4\Daily
Base path: C:\Users\PC1\OneDrive\A4
Data path : C:\Users\PC1\OneDrive\A4\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
Obsidian path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [6]:
file_name = "name-ttl.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file : {input_file}") 

Input file : C:\Users\PC1\OneDrive\A4\Data\name-ttl.csv


In [7]:
file_name = "price-hilo.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [8]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}")

Output file : C:\Users\PC1\OneDrive\A4\Data\price-hilo.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\price-hilo.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\price-hilo.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\price-hilo.csv


In [9]:
def cvt_str_to_flt(input_str):
    # Check for the special pattern and return 0
    if input_str == '(-)':
        return 0
    
    # Remove parentheses and percent sign
    cleaned_str = input_str.replace('(', '').replace(')', '').replace('%', '')

    # Convert to float
    try:
        return float(cleaned_str)
    except ValueError:
        # Handle the case where the conversion fails
        return None
 

In [10]:
def get_stock_data(in_df):

    in_df.columns = ['name']
    
    output_columns = ['name', 'price', 'today_low', 'today_high', 'year_low', 'year_high', 'percent', 'updated_at']
    # Initialize an empty DataFrame with specified columns
    out_df = pd.DataFrame(columns=output_columns)
    
    for index, row in in_df.iterrows():
        name = str(row['name']).upper()
        response = request.urlopen(url + name + '/price')
        html_page = BeautifulSoup(response, 'html.parser')
        
  #      price = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info').text.strip()        
  #      if (price == '-'):
  #          price = '0.00'        

        price_div = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info')
        if price_div:
            price = price_div.text.strip()
        else:
            print(f"Price div not found for {name}")
            price = '0.00'     
            
        # Try to find the <h3> tag with the class 'theme-success'
        h3_tag = html_page.find('h3', class_='theme-success')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-danger')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)    
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-normal')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)   
                
        percent = cvt_str_to_flt(percent_span)
                
        values_low = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder me-auto lh-1')
        today_low = values_low[0].text
        if (today_low == '-'):
            today_low = '0.00'
        
        values_high = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder lh-1')
        today_high = values_high[0].text
        if (today_high == '-'):
            today_high = '0.00'         
        
        year_low = values_low[1].text        
        year_high = values_high[1].text     
        
        # Find the div with the specific class, then find the span within it
        div = html_page.find('div', class_='price-detail-date d-flex align-items-center text-nowrap fs-12px')
        if div:
            span = div.find('span')
            if span:
                text = span.get_text()
#                 print(text)
            else:
                print("Span not found")
        else:
            print("Div not found")
            
        # Extract the date part from the string
        # Assuming the date is always in the format 'dd Mon yyyy'
        date_str = ' '.join(text.split()[2:5])

        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_str, '%d %b %Y')

        # Format the datetime object to 'yyyy-mm-dd'
        updated_at = date_obj.strftime('%Y-%m-%d')

#         print(updated_at)
        
        # Append row to the DataFrame and print the row
        out_df.loc[len(out_df)] = [name, price, today_low, today_high, year_low, year_high, percent, updated_at]
#       print(out_df.iloc[-1].to_string(header=False, index=False)) 
        print(','.join(map(str, out_df.iloc[-1])))
        sleep(2)
        
    return out_df  # return the final dataframe with all values populated

In [11]:
start_time = strftime("%I:%M %p")
print('Start at: ', start_time)

Start at:  09:11 PM


In [12]:
df = pd.read_csv(input_file, header=None)
out_df = get_stock_data(df)

ACE,1.30,1.29,1.30,1.23,1.38,-0.76,2026-01-05
ADVANC,321.00,315.00,325.00,259.00,320.00,2.56,2026-01-05
AEONTS,102.50,102.50,107.00,90.50,127.00,-4.65,2026-01-05
AH,12.50,12.50,12.60,10.00,16.30,-0.79,2026-01-05
AIE,1.03,1.03,1.04,0.72,1.80,0.0,2026-01-05
AIMIRT,10.90,10.80,10.90,9.20,10.90,0.0,2026-01-05
AIT,4.88,4.84,4.88,3.92,5.30,0.41,2026-01-05
AJ,1.94,1.92,2.02,1.54,4.06,3.19,2026-01-05
AMATA,15.90,15.90,16.80,11.20,28.75,-4.22,2026-01-05
ANAN,0.41,0.40,0.42,0.37,0.79,0.0,2026-01-05
AOT,54.50,53.00,55.00,26.75,60.25,2.83,2026-01-05
AP,8.85,8.70,8.90,5.60,9.25,2.91,2026-01-05
ASIAN,7.20,7.15,7.20,6.50,8.35,0.0,2026-01-05
ASK,7.10,7.05,7.30,6.20,10.30,-2.74,2026-01-05
ASP,2.02,2.00,2.04,1.81,2.80,0.0,2026-01-05
ASW,6.00,5.95,6.05,4.90,7.95,0.0,2026-01-05
AWC,2.10,2.06,2.16,1.55,3.56,-0.94,2026-01-05
BA,15.30,14.80,15.40,10.60,22.90,3.38,2026-01-05
BAM,8.05,7.95,8.15,5.35,8.60,0.63,2026-01-05
BANPU,4.92,4.92,5.05,4.00,6.05,0.0,2026-01-05
BAY,26.25,25.75,26.50,20.70,26.00,0.96,2026-0

In [13]:
end_time = strftime("%I:%M %p")
print('End at: ', end_time)

End at:  09:20 PM


In [14]:
out_df.query('price == "-"')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
25,BCT,-,0.00,0.00,43.50,70.00,0.0,2026-01-05


In [15]:
out_df.shape

(215, 8)

In [16]:
out_df = out_df[out_df['name'] != 'BCT']

In [17]:
out_df = out_df[out_df['name'] != 'LPF']

In [18]:
out_df = out_df[out_df['name'] != 'TYCN']

In [19]:
out_df = out_df[out_df['name'] != 'GRAMMY']

In [20]:
out_df = out_df[out_df['name'] != 'MST']

In [21]:
out_df = out_df[out_df['name'] != 'SPC']

In [22]:
out_df = out_df[out_df['name'] != 'SPRIME']

In [23]:
out_df = out_df[out_df['name'] != 'FPT']

In [24]:
out_df = out_df[out_df['name'] != 'GGC']

In [25]:
out_df = out_df[out_df['name'] != 'SKN']

In [26]:
out_df = out_df[out_df['name'] != 'KYE']

In [27]:
out_df = out_df[out_df['name'] != 'PDG']

In [28]:
out_df = out_df[out_df['name'] != 'PREB']

In [29]:
out_df = out_df[out_df['name'] != 'MBAX']

In [30]:
out_df = out_df[out_df['name'] != 'TYCN']

In [31]:
out_df.shape

(202, 8)

In [32]:
out_df['price'] = out_df['price'].astype('float')
out_df['today_low'] = out_df['today_low'].astype('float')
out_df['today_high'] = out_df['today_high'].astype('float')
out_df['year_low'] = out_df['year_low'].astype('float')
out_df['year_high'] = out_df['year_high'].astype('float')
out_df['percent'] = out_df['percent'].astype('float')

In [33]:
sorted_out_df = out_df.sort_values('percent')

In [34]:
sorted_out_df.query('today_low < year_low')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
186,TKN,3.90,3.86,3.96,3.88,9.25,-2.50,2026-01-05
34,BJC,14.10,14.00,14.50,14.10,24.60,-2.08,2026-01-05
213,WICE,2.24,2.22,2.28,2.28,3.96,-1.75,2026-01-05
116,MTC,31.00,30.75,32.00,31.00,49.75,-1.59,2026-01-05
100,LALIN,4.80,4.74,4.82,4.76,6.05,-0.41,2026-01-05
214,WORK,3.64,3.60,3.66,3.64,6.30,0.00,2026-01-05
208,VNG,1.78,1.73,1.78,1.74,3.06,1.14,2026-01-05


In [35]:
sorted_out_df.query('today_high > year_high')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
92,KBANK,194.50,194.50,198.00,145.00,197.00,0.00,2026-01-05
184,TISCO,111.00,110.00,111.50,95.00,111.00,0.45,2026-01-05
127,POPF,6.85,6.75,6.85,5.40,6.80,0.74,2026-01-05
177,TCAP,58.75,57.75,59.25,44.25,59.00,0.86,2026-01-05
20,BAY,26.25,25.75,26.50,20.70,26.00,0.96,2026-01-05
95,KKP,68.75,67.50,68.75,43.50,68.25,1.10,2026-01-05
154,SGP,7.95,7.70,8.00,5.00,7.90,1.27,2026-01-05
21,BBL,172.00,169.50,173.50,131.00,171.00,1.47,2026-01-05
199,TTB,2.06,2.04,2.10,1.74,2.08,1.98,2026-01-05
149,SCB,142.00,139.50,143.00,111.00,140.00,2.16,2026-01-05


In [36]:
out_df.query('today_high > year_high')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
1,ADVANC,321.00,315.00,325.00,259.00,320.00,2.56,2026-01-05
20,BAY,26.25,25.75,26.50,20.70,26.00,0.96,2026-01-05
21,BBL,172.00,169.50,173.50,131.00,171.00,1.47,2026-01-05
92,KBANK,194.50,194.50,198.00,145.00,197.00,0.00,2026-01-05
95,KKP,68.75,67.50,68.75,43.50,68.25,1.10,2026-01-05
127,POPF,6.85,6.75,6.85,5.40,6.80,0.74,2026-01-05
149,SCB,142.00,139.50,143.00,111.00,140.00,2.16,2026-01-05
154,SGP,7.95,7.70,8.00,5.00,7.90,1.27,2026-01-05
177,TCAP,58.75,57.75,59.25,44.25,59.00,0.86,2026-01-05
184,TISCO,111.00,110.00,111.50,95.00,111.00,0.45,2026-01-05


In [37]:
file_name = "Price-Hilo.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [38]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A4\Data\Price-Hilo.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\Price-Hilo.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\Price-Hilo.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\Price-Hilo.csv


In [39]:
out_df.to_csv(output_file, index=False)  
out_df.to_csv(god_file, index=False) 
out_df.to_csv(icd_file, index=False)  
out_df.to_csv(osd_file, index=False) 

In [40]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2026-01-05 21:20:10
